In [1]:
import torch
import torchvision
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import os
import datetime, time

In [2]:
os.getcwd()

'/home/student/Documents/Arpit_sir/SOLAR/CodeFiles/dhruv_git/Solar_Rooftop_Detection/Solar_Rooftop_Detection'

In [3]:
os.chdir("../")

In [4]:
### create image path lists
## images folder 
images_path = os.path.join(os.getcwd(),"Arial_images_1024_1024/images")
print(f"number of images are: {len(os.listdir(images_path))}")

## creating images list in which we are storing our images path
images = []

# for i in range(len(os.listdir(images_path))):
for i in range(10):
    images.append(f"{os.path.join(images_path,os.listdir(images_path)[i])}")


# print(images)

number of images are: 2880


In [5]:
len(images)

10

In [6]:
img_1 = images[0]
im = cv2.imread(img_1)

cv2.imshow("Demo-Image",im)
cv2.waitKey(10000)
cv2.destroyAllWindows()

In [7]:
cv2.destroyAllWindows()

In [8]:
## the same way lets create list of masks
masks_path = os.path.join(os.getcwd(),"Arial_images_1024_1024/masks")
print(f"number of masks are: {len(os.listdir(masks_path))}")

## creating masks list in which we are storing our masks path
masks = []

# for i in range(len(os.listdir(masks_path))):
for i in range(10):
    masks.append(f"{os.path.join(masks_path,os.listdir(masks_path)[i])}")

print(len(masks))

number of masks are: 2880
10


In [9]:
os.getcwd()

'/home/student/Documents/Arpit_sir/SOLAR/CodeFiles/dhruv_git/Solar_Rooftop_Detection'

In [10]:
os.listdir()

['.gitignore',
 'Solar_Rooftop_Detection',
 'sam_vit_b.pth',
 'ReadMe.MD',
 'logs',
 'test.ipynb',
 'Arial_images_1024_1024',
 'trial.ipynb',
 'checkpoints',
 '.git']

In [11]:
from Solar_Rooftop_Detection.get_bounding_box_cordinates import bounding_box

In [12]:
from Solar_Rooftop_Detection.SAM_logger import sam_logger

SAM_logger.py - 17-Feb-2025 20:56:18 - INFO - <module> - Line 25 - SAM model Logger has been started.


In [13]:
sam_logger.info("SAM Logger initialized")

2265001132.py - 17-Feb-2025 20:56:18 - INFO - <module> - Line 1 - SAM Logger initialized


In [14]:
## this dictionary contain image_path as key and as vlaues it store the list of bounding box cordinates.
bounding_boxes = {}

for i in range(len(images)):
    mask_img = cv2.imread(masks[i], cv2.IMREAD_GRAYSCALE)
    bounding_boxes[images[i]] = bounding_box(mask_img)

# print(len(bounding_boxes))

In [15]:
## now create custom Datset
from torch.utils.data import Dataset, DataLoader

class my_dataset(Dataset):
    def __init__(self, image_paths, mask_paths):
        self.image_paths = image_paths
        self.mask_paths = mask_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = cv2.imread(image_path)
        image = cv2.resize(image, (1024, 1024))
        
        # Convert image to RGB format
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        ## Load and preprocess the binary mask(1 channel)
        mask_path = self.mask_paths[idx]
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, (1024,1024))

        ## Convert image and mask to torch tensor 
        image = torch.tensor(image).permute(2,0,1).float()/255.0
        mask = torch.tensor(mask)

        return image, mask

In [16]:
dataset = my_dataset(images, masks)

In [17]:
### Now implement Data Loader
data_loader = DataLoader(dataset=dataset, batch_size=2, shuffle=True)
data_loader

In [18]:
from segment_anything import build_sam_vit_b

In [19]:
## run only when you dont have vit-b model in you local machine
# !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth -O sam_vit_b.pth

In [20]:
## initiallize the SAM model with vit-b configuration
model = build_sam_vit_b("/home/student/Documents/Arpit_sir/SOLAR/CodeFiles/dhruv_git/Solar_Rooftop_Detection/sam_vit_b.pth")
model

/home/student/Documents/Arpit_sir/solar_vnv/lib/python3.10/site-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.

Sam(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(in_features=3072, out_features=768, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): LayerNorm2d()
    )


In [21]:
## select device 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Sam(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(in_features=3072, out_features=768, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): LayerNorm2d()
    )


In [22]:
from torch.optim import Adam
import monai

In [23]:
# Initialize the optimizer and the loss function
optimizer = Adam(model.mask_decoder.parameters(), lr=1e-5, weight_decay=0)
#Try DiceFocalLoss, FocalLoss, DiceCELoss
seg_loss = monai.losses.DiceCELoss(sigmoid=True, squared_pred=True, reduction='mean')

In [24]:
from Solar_Rooftop_Detection.generate_isolated_masks import generate_isolated_mask

In [25]:
from torch.nn import functional as F

In [26]:
num_epochs = 20

In [27]:
model.named_parameters

<bound method Module.named_parameters of Sam(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(in_features=3072, out_features=768, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bi

In [28]:
# make sure we only compute gradients for mask decoder
for name, param in model.named_parameters():
    if name.startswith("image_encoder") or name.startswith("prompt_encoder"):
        param.requires_grad_(False)

In [29]:
checkpoint = "checkpoints"
os.makedirs(checkpoint, exist_ok=True)

In [30]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0001):
        """
        Args:
            patience (int): Number of epochs to wait before stopping if no improvement.
            min_delta (float): Minimum change in loss to be considered as improvement.
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float("inf")
        self.early_stop = False

    def __call__(self, val_loss, model, epoch):
        """Checks validation loss and determines if training should stop."""
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0  # Reset counter if improvement occurs

        else:
            self.counter += 1
            print(f"No improvement for {self.counter}/{self.patience} epochs...")

        if self.counter >= self.patience:
            print("Early stopping triggered. Training stopped.")
            self.early_stop = True

In [31]:
# Initialize early stopping
early_stopping = EarlyStopping(patience=5, min_delta=0.0001)

In [32]:
sam_logger.info("Training Started")

for epoch in range(num_epochs):
    # Train the model
    sam_logger.info(f"Epoch {epoch+1}/{num_epochs} started")

    running_loss = 0

    for img, mask in data_loader:
        image_ = img.to(device)
        mask_ = mask.to(device)
        # sam_logger.debug(f"images size : {image_.shape}, masks size : {mask_.shape}")

        for (image, mask) in zip(image_, mask_):
            
            # print(image.shape, mask.shape)

            ## getting the bounding box coordinates from masks 
            # print(mask.numpy())
            ## here we have to pass numpy array
            # print(mask.numpy().astype(np.uint8))
            
            bounding_box_corr = bounding_box(mask.detach().cpu().numpy().astype(np.uint8))
            sam_logger.info(f"Bounding Box Coordinates: {len(bounding_box_corr)}")

            # print(bounding_box_corr)


            for coordinates in bounding_box_corr:
                # print(f"points are : {coordinates}")
                one_isolated_mask = generate_isolated_mask(mask, coordinates)
                one_isolated_mask = one_isolated_mask.unsqueeze(0)
                one_isolated_mask = one_isolated_mask/255.0
                # print(f"Unique values of isolated masks are : ",torch.unique(one_isolated_mask))
                # print("isolated mask shape : ",one_isolated_mask.shape)
                
                # plt.imshow(one_isolated_mask.squeeze(0).numpy(), cmap="gray")
                # plt.show()
                
                
                sparse_embeddings, dense_embeddings = model.prompt_encoder(
                    points = None,
                    boxes = torch.tensor([coordinates]).unsqueeze(0).to(device),masks = None)
                
                low_res_mask, _ = model.mask_decoder(
                    image_embeddings = model.image_encoder(image.unsqueeze(0)),
                    image_pe = model.prompt_encoder.get_dense_pe(),
                    sparse_prompt_embeddings = sparse_embeddings,
                    dense_prompt_embeddings = dense_embeddings,
                    multimask_output = False
                )

                upsampled_masks = F.interpolate(low_res_mask, size = (1024, 1024), mode="bilinear", align_corners=False)

                ## calculate the loss
                # print("Upsampled mask shape : ",upsampled_masks.shape)
                # print("Unique values this tensor contain is : ", torch.unique(upsampled_masks))
                
                # print(upsampled_masks)
                loss = seg_loss(upsampled_masks, one_isolated_mask.unsqueeze(0).float())
                
                # backward pass (compute gradients of parameters w.r.t. loss)
                optimizer.zero_grad()

                loss.backward()
                
                # Optimize
                optimizer.step()
                
                if len(bounding_box_corr) > 0:
                    running_loss += loss.item() / len(bounding_box_corr)
                
                # print(f"runing loss: {running_loss}")
                sam_logger.info(f"Current running loss: {running_loss:.4f}")



                # to see the output and actual masks
                # copy_up_sampled = upsampled_masks.clone()

                # plt.imshow(copy_up_sampled.squeeze(0).squeeze(0).detach().numpy(),cmap="gray")
                # plt.tight_layout()
                # plt.show()

    
    # print(f"epoch {epoch}, loss: {running_loss / len(data_loader)}")
    sam_logger.info(f"Epoch {epoch+1} completed. Loss: {running_loss / len(data_loader):.4f}")
    
    early_stopping(running_loss, model, epoch)
    if early_stopping.early_stop:
        sam_logger.error(f"Early Stoping : Epoch {epoch+1} completed. Loss: {running_loss / len(data_loader):.4f}")
        sam_logger.error(f"TRAINING STOPPED EARLY")
        break  # Stop training if early stopping is triggered

    
    
    ## save the model checkpoint every 10 epoch
    
    if (epoch + 1) % 10 == 0:
        current_date = datetime.datetime.now().strftime("%d_%m_%Y")
        os.makedirs(checkpoint, exist_ok=True)
        checkpoint_path = os.path.join(checkpoint, f"sam_model_{epoch + 1}_{current_date}.pth")
        torch.save(model.state_dict(), checkpoint_path)
        sam_logger.info(f"Model checkpoint saved at {checkpoint_path}")


1470904569.py - 17-Feb-2025 20:56:19 - INFO - <module> - Line 1 - Training Started
1470904569.py - 17-Feb-2025 20:56:19 - INFO - <module> - Line 5 - Epoch 1/20 started
1470904569.py - 17-Feb-2025 20:56:19 - INFO - <module> - Line 24 - Bounding Box Coordinates: 0
1470904569.py - 17-Feb-2025 20:56:19 - INFO - <module> - Line 24 - Bounding Box Coordinates: 4
1470904569.py - 17-Feb-2025 20:56:24 - INFO - <module> - Line 74 - Current running loss: 0.0123
1470904569.py - 17-Feb-2025 20:56:28 - INFO - <module> - Line 74 - Current running loss: 0.0426
1470904569.py - 17-Feb-2025 20:56:33 - INFO - <module> - Line 74 - Current running loss: 0.0747
1470904569.py - 17-Feb-2025 20:56:38 - INFO - <module> - Line 74 - Current running loss: 0.1318
1470904569.py - 17-Feb-2025 20:56:38 - INFO - <module> - Line 24 - Bounding Box Coordinates: 2
1470904569.py - 17-Feb-2025 20:56:42 - INFO - <module> - Line 74 - Current running loss: 0.2378
1470904569.py - 17-Feb-2025 20:56:46 - INFO - <module> - Line 74 - 

In [31]:
final_model_checkpoint = "checkpoints/Final_Models"
os.makedirs(checkpoint, exist_ok=True)

In [32]:
current_date = f"images_{len(images)}_epoch_{num_epochs}_"+datetime.datetime.now().strftime("%d_%m_%Y")+".pth"
final_model_path = os.path.join(final_model_checkpoint, current_date)

In [33]:
torch.save(model.state_dict(), final_model_path)